In [ ]:
# pip install requests bs4 lxml xlsxwriter

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 2.9 MB/s eta 0:00:0000:0100:01
Note: you may need to restart the kernel to use updated packages.


In [1]:
import csv
import requests
from bs4 import BeautifulSoup
import time
from urllib.parse import urljoin
import pandas as pd
import re


In [ ]:
def CleanData(InfoString):
    """Clean the data by removing unwanted characters"""
    InfoString = InfoString.replace('\n', '',)  
    InfoString = InfoString.replace('\r', '',)  
    InfoString = InfoString.replace(' ', '',)
    InfoString = InfoString.replace('年', '-', 1)  
    InfoString = InfoString.replace('月', '-', 1)
    InfoString = InfoString.replace('日', '', 1)
    return InfoString

def ExtractWeatherForCity(city):
    """Scrape weather data for the specified city and append to the same CSV file"""
    # Define the base URL for the weather history page for different cities
    url = f'http://www.tianqihoubao.com/lishi/{city}.html'
    
    # Open the same CSV file to append data from different cities
    output_file = 'Weather_Data_All_Cities.csv'
    with open(output_file, mode='a', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        
        # If it's the first city, write the header
        if file.tell() == 0:  # Check if file is empty, i.e., first write
            writer.writerow(['Date', 'City', 'Weather Condition', 'Temperature', 'Wind Force and Direction'])  # CSV headers
        
        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.106 Safari/537.36',
            'Connection': 'close'
        }
        html = requests.get(url, headers=headers)
        html.encoding = 'gb2312'  # Correctly set the encoding to match the website
        bsObj = BeautifulSoup(html.text, 'lxml')

        # Find the weather data for the city
        year_weather_data = bsObj.find_all('div', class_="box pcity")  # Weather data for all years
        year = 2010
        for quarter_data in year_weather_data:  # Iterate through quarterly data
            year += 1
            if year <= 2021:  # Restrict to data up to 2020
                quarter_list = quarter_data.find_all('ul')  # Quarterly data
                for month_data in quarter_list:  # Iterate through months in a quarter
                    month_links = month_data.find_all('a')  # Extract links for each month
                    for link in month_links:
                        month_url = urljoin('http://www.tianqihoubao.com', link['href'])
                        try:
                            month_html = requests.get(month_url, headers=headers)
                            month_html.encoding = 'gb2312'  # Correct encoding
                            month_bsObj = BeautifulSoup(month_html.text, 'lxml')
                            month_rows = month_bsObj.find_all('tr')  # Rows of monthly data
                            for i, day_row in enumerate(month_rows):
                                if i == 0:  # Skip the header row
                                    continue
                                day_data = [CleanData(td.get_text()) for td in day_row.find_all('td')]
                                if len(day_data) == 4:
                                    day_data.insert(1, city)  # Insert city name at the second column
                                    writer.writerow(day_data)  # Write data to CSV file
                        except Exception as e:  # Handle request rate limit
                            print(f"Error scraping {city} data for month: {link['href']}, retrying...")
                            time.sleep(5)
                            continue
                print(f'Finished scraping data for {year}...')
            else:
                break
    print(f'Scraping data for {city} completed!')
    return

# Example: Scrape weather data for Beijing, Shanghai, and Guangzhou
ExtractWeatherForCity('beijing')
ExtractWeatherForCity('shanghai')
ExtractWeatherForCity('lasa')
ExtractWeatherForCity('haikou')

Finished scraping data for 2011...
Finished scraping data for 2012...
Finished scraping data for 2013...
Finished scraping data for 2014...
Finished scraping data for 2015...
Finished scraping data for 2016...
Finished scraping data for 2017...
Finished scraping data for 2018...
Finished scraping data for 2019...
Finished scraping data for 2020...
Finished scraping data for 2021...
Scraping data for beijing completed!
Finished scraping data for 2011...
Finished scraping data for 2012...
Finished scraping data for 2013...
Finished scraping data for 2014...
Finished scraping data for 2015...
Finished scraping data for 2016...
Finished scraping data for 2017...
Finished scraping data for 2018...
Finished scraping data for 2019...
Finished scraping data for 2020...
Finished scraping data for 2021...
Scraping data for shanghai completed!
Finished scraping data for 2011...
Finished scraping data for 2012...
Finished scraping data for 2013...
Finished scraping data for 2014...
Finished scrapi

In [4]:

data = pd.read_csv('Weather_Data_All_Cities.csv')
data['Date'] = pd.to_datetime(data['Date'], format='%Y-%m-%d')
data[['Day Weather', 'Night Weather']] = data['Weather Condition'].str.split('/', expand=True)

data[['Day Temp', 'Night Temp']] = data['Temperature'].str.split('/', expand=True)

data['Day Temp'] = data['Day Temp'].str.replace('℃', '').replace('', pd.NA)
data['Night Temp'] = data['Night Temp'].str.replace('℃', '').replace('', pd.NA)

data['Day Temp'] = pd.to_numeric(data['Day Temp'], errors='coerce')  # 'coerce'会将无法转换的值设为NaN
data['Night Temp'] = pd.to_numeric(data['Night Temp'], errors='coerce')

wind_force_pattern = r'(\d+-\d+级|\d+级|≤\d+级)'

# 风向的正则：匹配包含'风'的字符串（例如：北风、东南风、无持续风向等）
wind_direction_pattern = r'([东南西北]+风|无持续风向)'

# 分离风力和风向信息
def extract_wind_info(wind_data):
    # 使用斜杠来分开白天和夜间的风力风向数据
    if pd.isnull(wind_data):
        return ['Unknown', 'Unknown', 'Unknown', 'Unknown']

    parts = wind_data.split('/')  # 通过斜杠分开白天和夜间的数据
    if len(parts) == 2:  # 如果有两部分数据（即白天和夜间）
        day_data = parts[0]
        night_data = parts[1]
    else:
        day_data = parts[0]
        night_data = '无持续风向'  # 默认值，如果夜间没有数据

    # 提取白天和夜间风力与风向
    day_wind_direction = re.search(wind_direction_pattern, day_data)
    night_wind_direction = re.search(wind_direction_pattern, night_data)
    day_wind_force = re.search(wind_force_pattern, day_data)
    night_wind_force = re.search(wind_force_pattern, night_data)

    day_wind_direction = day_wind_direction.group(0) if day_wind_direction else 'Unknown'
    night_wind_direction = night_wind_direction.group(0) if night_wind_direction else 'Unknown'
    day_wind_force = day_wind_force.group(0) if day_wind_force else 'Unknown'
    night_wind_force = night_wind_force.group(0) if night_wind_force else 'Unknown'

    return [day_wind_direction, night_wind_direction, day_wind_force, night_wind_force]

# 通过apply方法将提取操作应用到每一行的'Wind Force and Direction'列
data[['Day Wind Direction', 'Night Wind Direction', 'Day Wind Force', 'Night Wind Force']] = data['Wind Force and Direction'].apply(
    extract_wind_info).apply(pd.Series)

# 查看处理后的数据
print(data.head())

        Date     City Weather Condition Temperature Wind Force and Direction  \
0 2011-01-01  beijing               晴/晴      0℃/-9℃        无持续风向≤3级/无持续风向≤3级   
1 2011-01-02  beijing              多云/阴     -2℃/-7℃        无持续风向≤3级/无持续风向≤3级   
2 2011-01-03  beijing               晴/晴      1℃/-8℃          北风3-4级/无持续风向≤3级   
3 2011-01-04  beijing               晴/晴    -1℃/-11℃        无持续风向≤3级/无持续风向≤3级   
4 2011-01-05  beijing               晴/晴     -1℃/-8℃            北风4-5级/北风3-4级   

  Day Weather Night Weather  Day Temp  Night Temp Day Wind Direction  \
0           晴             晴       0.0        -9.0              无持续风向   
1          多云             阴      -2.0        -7.0              无持续风向   
2           晴             晴       1.0        -8.0                 北风   
3           晴             晴      -1.0       -11.0              无持续风向   
4           晴             晴      -1.0        -8.0                 北风   

  Night Wind Direction Day Wind Force Night Wind Force  
0                无持续风向       